# Introduction to SDC



[//]: # (Comment: Page 5 of source document)
## 1.1	Prerequisites (TBD)

## 1.2	The Lives of Data (TBD)


## 1.3	What is Structured Data Capture (SDC)?


**Structured Data Capture (SDC)** 
is a new technology that creates interoperable, computer-readable definitions for standardized **Question/Answer Sets (QAS)** 
in **Data Entry Forms (DEFs)**.

SDC standardizes the creation and management of DEF QAS items throughout the data lifecycle. The SDC data lifecycle begins with the design of QAS items in DEFs and the inclusion of DEFs in DEF libraries.  SDC standardizes the representation of QAS items in a DEF on a computer screen and defines the behavior of DEFs during user-DEF interactions.  

After data is captured in an SDC DEF, the SDC model standardizes the transmission and redisplay of captured DEF user responses, as well as re-editing of the original DEF-captured data, and re-transmission of the DEF and its data to other recipients.  SDC is also capable of creating specifications for reports based on the captured DEF data, and provides recommendations for the storage and querying of DEF-captured data.

This document describes design of DEFs using the SDC model and touches upon several of the other data lifecycle activities.  As a general organizational pattern for this document, we will introduce many topics at a high level early, and then come back to them in progressively more detail.

## 1.4	SDC’s History and Objectives

The [SDC project](https://oncprojectracking.healthit.gov/wiki/display/TechLabSC/SDC+Home) was initiated by the Office of the National Coordinator for Health Information Technology (ONC) in early 2013 through its Standards and Interoperability (S&I) Framework initiative.  Independent SDC-like technologies had emerged previously, but no technology-agnostic standard was available.  SDC’s technical workgroups have focused on creating standards by which interoperable forms are defined, rendered, populated and exchanged. 

SDC was developed in cooperation with [Integrating the Healthcare Enterprise (IHE)](https://en.wikipedia.org/wiki/Integrating_the_Healthcare_Enterprise), a standards organization which emphasizes the interoperability of healthcare information technology (HIT) systems, with a focus on combining constrained standards into profiles for interoperable data transmission. IHE gathers case requirements, identifies available standards, and develops technical guidelines which technical professionals can implement. IHE also hosts yearly “Connectathons” in several countries and stages “interoperability showcases” at [HIMSS](http://www.himss.org/) meetings, at which vendors assemble to demonstrate the interoperability of their products. The SDC workgroup has participated yearly in these IHE activities since 2014.

In keeping with ONC’s role as a standards incubator, in April 2017, ONC transitioned the SDC project to an IHE “community led” project in which many organizations continue to evolve the work incubated by ONC.  The [IHE SDC Profile](http://ihe.net/uploadedFiles/Documents/QRPH/IHE_QRPH_Suppl_SDC.pdf) describes the SDC features and data exchange using the [Request Form for Data Capture standard](https://profiles.ihe.net/ITI/TF/Volume1/ch-17.html). Other transport formats include the [North American Association of Central Cancer Registries (NAACCR) Volume V](https://www.naaccr.org/pathology-laboratory-electronic-reporting/) and [SDC on FHIR](http://hl7.org/fhir/uv/ihe-sdc-ecc/).

[//]: # (Comment: Page 6 of source document)
# 2	Overview of SDC Principles
The SDC model is defined by the SDC **XML Schema**, which provides the definitions for creating XML **Form Design File**s (**FDF**s).  FDFs are thus **XML instance documents** that conform to the SDC Schema definition. An FDF provides a standardized definition of QAS content and user-interaction behavior for a single DEF, and is designed to be transformed automatically into an SDC-based DEF.  The QAS content inside an FDF is intended to support reusable QAS blocks called Data Elements (DEs), which are discussed later.  Users’ responses that are captured in the DEF are added to the FDF XML (now called an FDF-Response File [FDF R]), and then the responses are transmitted to one or more endpoints. 

## 2.1	SDC Design Principles
A brief review of SDC’s functional and technical requirements will clarify the reason for many design decisions.  
- The primary use-cases for SDC are to:
    - Create interoperable clinical data-entry standards for FDFs and DEFs[^1] 
    - Enable downstream uses (e.g., quality assessments and public health analytics) of the captured standardized data
- FDFs, not terminologies or Common Data Elements (CDEs)[^2] , are the primary source of context-sensitive semantics.[^3]  
- SDC uses a single computer-readable information model to standardize DE content in the FDF and DEF.  Thus, DEF content is standardized before any user response data is captured by a DEF or data storage device.
- SDC “Form Fillers” (see below) are built to render any SDC-conformant FDF, regardless of the FDF content.
- SDC also supports the definition of report formats, distinct from the DEF layout.
- SDC uses open-source technical standards to define technology-agnostic blueprints for DEF design.
    - SDC has no preferred programming languages.
    - SDC uses industry-standard XML-based mechanisms for the creation and interoperable exchange of FDFs and user responses.
    - SDC uses an interoperable, computer-readable, Schema-defined, XML format to represent the SDC information model, and that allows a computer to build and exchange a wide variety of standardized DEFs.
    - SDC provides an interoperability mechanism for saving and transporting user-entered data inside its original FDF, with 100% round-trip fidelity.[^4] 
    - SDC includes XHTML support for formatting HTML-based rich text.[^5] 

---

[^1]: The format for the FDF XML file is defined by an XML Schema.  The FDF XML defines the information (e.g., Questions and answer choices) that must be displayed in a computer screen (DEF), and also describes essential features of the DEF behavior when the user is interacting with the form on a computer screen.

[^2]: A CDE is a DE designed for widespread use and is housed in a CDE registry.

[^3]: In many cases, CDEs and terminologies (e.g., SNOMED CT) are insufficient to describe the full nuanced meaning of a captured response in a DEF and consideration of the DEF context is required to fully understand the captured user responses.  However, annotation of FDF-DEs with CDEs and terminologies is useful in many cases, such as when aggregating data from SDC DEFs and other sources.  

[^4]: i.e., back and forth exchange of one SDC dataset (the data from an SDC DEF, inside an FDF-R) across multiple nodes, with no loss of data or context.  The original DEF and its user-entered data will be visible and unchanged regardless of how many times it has been transmitted across multiple nodes.

[^5]: This allows the use of complex rich text (e.g., fonts, special text formatting) alongside the equivalent plain (unformatted) text in FDFs.  Although many DEFs appear “better” with rich text, some systems cannot support it.  Also, many data transmission standards do not natively support rich text of any sort.


[//]: # (Comment: Page 7 of source document)
## 2.2	SDC Actors
In the simplest SDC model, there are three primary software actors within the SDC ecosystem. The three primary software actors are as follows:

* Form Managers (FMs)
* Form Fillers (FFs)
* Form Receivers (FRs)

Each of these actors serves as a distinct node in the SDC transaction network. 

### Form Managers
FMs are responsible for storing Form Definition Files (FDFs) in a repository. They promptly transmit these FDFs upon receiving requests from Form Fillers (FFs). FMs are also tasked with addressing user authorization and authentication, generating instance IDs, and enforcing instance ID and version control (which will be discussed later).

### Form Fillers
Form Fillers are software applications that perform the following tasks:
a. Retrieve an FDF file from a Form Manager.
i. Alternatively, the FF may retrieve an FDF transformed into HTML or obtain a URL that directs to a server hosting the SDC HTML.
b. Convert the XML content into a Document Element Form (DEF) using various programming languages and methodologies. For instance, the XSLT language might be used to transform the FDF into an HTML web page. Users interact with the DEF within the FF software.
c. Capture and validate user-entered responses within the DEF.
d. Implement both implicit9 and explicit10 rules that dictate the behaviors and actions of the forms in response to user interactions.
e. Store and/or transmit the captured response data contained inside the original FDF. As noted earlier, an FDF containing user responses is called an FDF-R. SDC response data is transmitted as an FDF-R to one or more actors called Form Receivers (FRs).

### Form Receivers
Form Receivers receive the SDC response data (in an FDF-R file) from the Form Filler and process the data
according to the FR’s needs. FRs are responsible for storing the captured SDC data as native SDC XML and/or
transformed into some other storage format (e.g., relational database tables). FRs may also be responsible for
validation of forms via the SDC Schema, SDC rules, and external rules encoded in formats such as
Schematrons. Transaction logging, validation, error reporting to the FF, version control, patient matching,
authentication and authorization are other potential requirements for FRs.

This delineates the roles and responsibilities of the primary software actors in the simplest SDC model.